In [1]:
import pandas as pd 
import csv
import regex as re
import json

In [2]:
df = pd.read_csv('MAPA_Travaux.csv', encoding = 'utf-8', delimiter = ';')

C:\Users\couss\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (5,13) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df

,idweb,annonce_reference_schema_v110,objet,famille,code_departement,code_departement_prestation,famille_libelle,dateparution,datefindiffusion,datelimitereponse,...,descripteur_libelle,type_marche,type_marche_facette,type_avis,annonce_lie,annonces_anterieures,annonces_anterieures_schema_v110,source_schema,GESTION,DONNEES
0,17-10008,NaN,Réalisation d'un schéma directeur des systèmes...,MAPA,56,NaN,Marchés <90 k€ (MAPA),2017-01-20,2017-02-20,NaN,...,Etude;Informatique (prestations de services),"SERVICES,TRAVAUX","Services,Travaux","5,1,",NaN,NaN,NaN,Boamp_v230.xsd,"{""INDEXATION"": {""RESUME_OBJET"": ""Réalisation d...","{""OBJET"": {""VARIANTES_NON"": """", ""LIEU_EXEC_LIV..."
1,17-20008,NaN,appel à candidaturesmarché de conception réali...,MAPA,67,NaN,Marchés <90 k€ (MAPA),2017-02-10,2017-03-06,2017-03-06T12:00:00+01:00,...,Chauffage (travaux),"SERVICES,TRAVAUX","Services,Travaux","5,1,",NaN,NaN,NaN,Boamp_v230.xsd,"{""INDEXATION"": {""DATE_FIN_DIFFUSION"": ""2017-03...","{""OBJET"": {""OBJET_COMPLET"": ""Appel à candidatu..."
2,17-66902,NaN,MARCHE DE TRAVAUX RELATIF A L'AMENAGEMENT DE B...,MAPA,75,NaN,Marchés <90 k€ (MAPA),2017-05-12,2017-06-10,2017-06-09T17:00:00+02:00,...,Electricité (travaux),"SERVICES,TRAVAUX","Services,Travaux","5,1,",NaN,NaN,NaN,Boamp_v230.xsd,"{""INDEXATION"": {""DATE_FIN_DIFFUSION"": ""2017-06...","{""OBJET"": {""VARIANTES_OUI"": """", ""LIEU_EXEC_LIV..."
3,15-91869,NaN,Entretien du réseau d'eaux pluviales de la vil...,MAPA,13,13,Marchés <90 k€ (MAPA),2015-06-16,2015-07-28,2015-07-28T13:00:00+02:00,...,Assainissement;Voirie et réseaux divers,"SERVICES,TRAVAUX","Services,Travaux","5,1,",NaN,NaN,NaN,Boamp_v230.xsd,"{""INDEXATION"": {""DATE_FIN_DIFFUSION"": ""2015-07...","{""OBJET"": {""OBJET_COMPLET"": ""Entretien du rése..."
4,15-106209,NaN,Travaux de désamiantage,MAPA,93,93,Marchés <90 k€ (MAPA),2015-07-08,2015-07-20,2015-07-20T13:00:00+02:00,...,Amiante,"SERVICES,TRAVAUX","Services,Travaux","5,1,",NaN,NaN,NaN,Boamp_v230.xsd,"{""INDEXATION"": {""DATE_FIN_DIFFUSION"": ""2015-07...","{""OBJET"": {""VARIANTES_NON"": """", ""LIEU_EXEC_LIV..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144873,21-150007,NaN,"relance du lot plomberie, chauffage, vmc pour ...",MAPA,28,NaN,Marchés <90 k€ (MAPA),2021-11-10,2021-12-02,2021-12-02T12:00:00+01:00,...,Plomberie (travaux),TRAVAUX,Travaux,"5,1,",NaN,NaN,NaN,Boamp_v230.xsd,"{""INDEXATION"": {""DATE_FIN_DIFFUSION"": ""2021-12...","{""OBJET"": {""VARIANTES_NON"": """", ""LIEU_EXEC_LIV..."
144874,21-150978,NaN,TRAVAUX DE SIGNALISATION HORIZONTALE / MARQUAG...,MAPA,13,NaN,Marchés <90 k€ (MAPA),2021-11-15,2021-12-17,2021-12-17T12:00:00+01:00,...,Signalisation,TRAVAUX,Travaux,"5,1,",NaN,NaN,NaN,Boamp_v230.xsd,"{""INDEXATION"": {""DATE_FIN_DIFFUSION"": ""2021-12...","{""OBJET"": {""VARIANTES_NON"": """", ""LIEU_EXEC_LIV..."
144875,21-151940,NaN,RESTAURATION DES DÉCORS INTÉRIEURS DE L'EGLISE...,MAPA,"11,31,9,34,66",NaN,Marchés <90 k€ (MAPA),2021-11-16,2021-12-16,2021-12-17T11:00:00+01:00,...,Clocher;Décoration (travaux);Electricité (trav...,TRAVAUX,Travaux,"5,1,",NaN,NaN,NaN,Boamp_v230.xsd,"{""INDEXATION"": {""DATE_FIN_DIFFUSION"": ""2021-12...","{""OBJET"": {""LOTS"": {""LOT"": [{""NUM"": ""1"", ""DESC..."
144876,21-157312,NaN,Aménagement des locaux Police municipale et Ba...,MAPA,37,NaN,Marchés <90 k€ (MAPA),2021-11-29,2021-12-30,2021-12-30T23:00:00+01:00,...,Bâtiment,TRAVAUX,Travaux,"5,1,",NaN,NaN,NaN,Boamp_v230.xsd,"{""INDEXATION"": {""DATE_FIN_DIFFUSION"": ""2021-12...","{""OBJET"": {""LOTS"": {""LOT"": [{""NUM"": ""2"", ""DESC..."


In [4]:
df.columns

Index(['idweb', 'annonce_reference_schema_v110', 'objet', 'famille',
       'code_departement', 'code_departement_prestation', 'famille_libelle',
       'dateparution', 'datefindiffusion', 'datelimitereponse', 'nomacheteur',
       'perimetre', 'type_procedure', 'soustype_procedure',
       'procedure_libelle', 'procedure_categorise', 'nature', 'sousnature',
       'nature_libelle', 'sousnature_libelle', 'nature_categorise',
       'nature_categorise_libelle', 'criteres', 'marche_public_simplifie',
       'marche_public_simplifie_label', 'etat', 'descripteur_code',
       'descripteur_libelle', 'type_marche', 'type_marche_facette',
       'type_avis', 'annonce_lie', 'annonces_anterieures',
       'annonces_anterieures_schema_v110', 'source_schema', 'GESTION',
       'DONNEES'],
      dtype='object')

In [5]:
df['datefindiffusion'].unique()

array(['2017-02-20', '2017-03-06', '2017-06-10', ..., '2021-12-05',
       '2022-01-08', '2021-12-02'], dtype=object)

In [6]:
# Tous les états sont en 'initial'
# annonce_lie a l'air pareil que idweb
df = df.drop(['annonce_reference_schema_v110','famille','soustype_procedure','procedure_libelle','procedure_categorise','sousnature'
              ,'sousnature_libelle','nature_categorise','nature_categorise_libelle',
             'marche_public_simplifie','marche_public_simplifie_label','type_marche_facette',
             'annonces_anterieures_schema_v110','source_schema'], axis=1)

In [7]:
df

,idweb,objet,code_departement,code_departement_prestation,famille_libelle,dateparution,datefindiffusion,datelimitereponse,nomacheteur,perimetre,...,criteres,etat,descripteur_code,descripteur_libelle,type_marche,type_avis,annonce_lie,annonces_anterieures,GESTION,DONNEES
0,17-10008,Réalisation d'un schéma directeur des systèmes...,56,NaN,Marchés <90 k€ (MAPA),2017-01-20,2017-02-20,NaN,Pontivy Communauté,CMP-2006,...,NaN,INITIAL,"118,163",Etude;Informatique (prestations de services),"SERVICES,TRAVAUX","5,1,",NaN,NaN,"{""INDEXATION"": {""RESUME_OBJET"": ""Réalisation d...","{""OBJET"": {""VARIANTES_NON"": """", ""LIEU_EXEC_LIV..."
1,17-20008,appel à candidaturesmarché de conception réali...,67,NaN,Marchés <90 k€ (MAPA),2017-02-10,2017-03-06,2017-03-06T12:00:00+01:00,Sibar,CMP-2006,...,NaN,INITIAL,61,Chauffage (travaux),"SERVICES,TRAVAUX","5,1,",NaN,NaN,"{""INDEXATION"": {""DATE_FIN_DIFFUSION"": ""2017-03...","{""OBJET"": {""OBJET_COMPLET"": ""Appel à candidatu..."
2,17-66902,MARCHE DE TRAVAUX RELATIF A L'AMENAGEMENT DE B...,75,NaN,Marchés <90 k€ (MAPA),2017-05-12,2017-06-10,2017-06-09T17:00:00+02:00,AGCE POUR L ENST FR A L ETRANGER,CMP-2006,...,NaN,INITIAL,105,Electricité (travaux),"SERVICES,TRAVAUX","5,1,",NaN,NaN,"{""INDEXATION"": {""DATE_FIN_DIFFUSION"": ""2017-06...","{""OBJET"": {""VARIANTES_OUI"": """", ""LIEU_EXEC_LIV..."
3,15-91869,Entretien du réseau d'eaux pluviales de la vil...,13,13,Marchés <90 k€ (MAPA),2015-06-16,2015-07-28,2015-07-28T13:00:00+02:00,Mairie de Pélissanne,CMP-2006,...,NaN,INITIAL,"18,366",Assainissement;Voirie et réseaux divers,"SERVICES,TRAVAUX","5,1,",NaN,NaN,"{""INDEXATION"": {""DATE_FIN_DIFFUSION"": ""2015-07...","{""OBJET"": {""OBJET_COMPLET"": ""Entretien du rése..."
4,15-106209,Travaux de désamiantage,93,93,Marchés <90 k€ (MAPA),2015-07-08,2015-07-20,2015-07-20T13:00:00+02:00,Université Paris 8,CMP-2006,...,NaN,INITIAL,24,Amiante,"SERVICES,TRAVAUX","5,1,",NaN,NaN,"{""INDEXATION"": {""DATE_FIN_DIFFUSION"": ""2015-07...","{""OBJET"": {""VARIANTES_NON"": """", ""LIEU_EXEC_LIV..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144873,21-150007,"relance du lot plomberie, chauffage, vmc pour ...",28,NaN,Marchés <90 k€ (MAPA),2021-11-10,2021-12-02,2021-12-02T12:00:00+01:00,COMMUNE CLOYES LES TROIS RIVIERES,CMP-2006,...,NaN,INITIAL,270,Plomberie (travaux),TRAVAUX,"5,1,",NaN,NaN,"{""INDEXATION"": {""DATE_FIN_DIFFUSION"": ""2021-12...","{""OBJET"": {""VARIANTES_NON"": """", ""LIEU_EXEC_LIV..."
144874,21-150978,TRAVAUX DE SIGNALISATION HORIZONTALE / MARQUAG...,13,NaN,Marchés <90 k€ (MAPA),2021-11-15,2021-12-17,2021-12-17T12:00:00+01:00,COMMUNE DE SAINT MITRE LES REMPARTS,CMP-2006,...,NaN,INITIAL,322,Signalisation,TRAVAUX,"5,1,",NaN,NaN,"{""INDEXATION"": {""DATE_FIN_DIFFUSION"": ""2021-12...","{""OBJET"": {""VARIANTES_NON"": """", ""LIEU_EXEC_LIV..."
144875,21-151940,RESTAURATION DES DÉCORS INTÉRIEURS DE L'EGLISE...,"11,31,9,34,66",NaN,Marchés <90 k€ (MAPA),2021-11-16,2021-12-16,2021-12-17T11:00:00+01:00,MAIRIE DE FANJEAUX,CMP-2006,...,NaN,INITIAL,"80,85,105,195,264,308",Clocher;Décoration (travaux);Electricité (trav...,TRAVAUX,"5,1,",NaN,NaN,"{""INDEXATION"": {""DATE_FIN_DIFFUSION"": ""2021-12...","{""OBJET"": {""LOTS"": {""LOT"": [{""NUM"": ""1"", ""DESC..."
144876,21-157312,Aménagement des locaux Police municipale et Ba...,37,NaN,Marchés <90 k€ (MAPA),2021-11-29,2021-12-30,2021-12-30T23:00:00+01:00,Ville de Saint-Avertin,CMP-2006,...,NaN,INITIAL,33,Bâtiment,TRAVAUX,"5,1,",NaN,NaN,"{""INDEXATION"": {""DATE_FIN_DIFFUSION"": ""2021-12...","{""OBJET"": {""LOTS"": {""LOT"": [{""NUM"": ""2"", ""DESC..."


In [8]:
first_line = df['DONNEES'][0]
first_line

'{"OBJET": {"VARIANTES_NON": "", "LIEU_EXEC_LIVR": {"VILLE": "Pontivy", "CP": "56300", "ADRESSE": "Territoire de pontivy communauté"}, "OBJET_COMPLET": "Réalisation d\'un schéma directeur des systèmes d\'information intercommunal", "DIV_EN_LOTS": {"NON": ""}, "CARACTERISTIQUES": {"QUANTITE": "Réalisation d\'un schéma directeur des systèmes d\'information intercommunal", "PRINCIPALES": "Délai d\'exécution : 36 mois Variantes non autorisées.Accord-cadre à bons de commande et à tranche optionnelle. Marché à prix mixtes : forfaitaires et unitaires à bons de commande.Pontivy Communauté figurant au titre des pilotes du projet national 100% demat des marchés publics, conformément à l\'article 40 du Décret 2016-360, la collectivité impose pour ce marché le mode de transmission des offres par voie électronique notamment via le dispositif MPS permettant de réduire les informations à transmettre. Ce marché fait partie du dispositif \\" marché public simplifié \\" (MPS) dans le cadre du programme 

In [9]:
# On enlève le objet et les {} (au début et à la fin) car inutile
first_line = first_line.replace('{"OBJET": ','')
first_line=first_line[:-1]

In [10]:
import re
# on sépare les éléments en fonction de {} 
# pour commencer à voir les différents dictionnaires et clés/valeures
first_line = re.split('[{}]', first_line)
first_line

['',
 '"VARIANTES_NON": "", "LIEU_EXEC_LIVR": ',
 '"VILLE": "Pontivy", "CP": "56300", "ADRESSE": "Territoire de pontivy communauté"',
 ', "OBJET_COMPLET": "Réalisation d\'un schéma directeur des systèmes d\'information intercommunal", "DIV_EN_LOTS": ',
 '"NON": ""',
 ', "CARACTERISTIQUES": ',
 '"QUANTITE": "Réalisation d\'un schéma directeur des systèmes d\'information intercommunal", "PRINCIPALES": "Délai d\'exécution : 36 mois Variantes non autorisées.Accord-cadre à bons de commande et à tranche optionnelle. Marché à prix mixtes : forfaitaires et unitaires à bons de commande.Pontivy Communauté figurant au titre des pilotes du projet national 100% demat des marchés publics, conformément à l\'article 40 du Décret 2016-360, la collectivité impose pour ce marché le mode de transmission des offres par voie électronique notamment via le dispositif MPS permettant de réduire les informations à transmettre. Ce marché fait partie du dispositif \\" marché public simplifié \\" (MPS) dans le cadr

In [11]:
#on va enlever les éléments vides/ sans infos
for elm in first_line:
    if not re.search(':("|\s")',elm):
        print(elm)
        first_line.remove(elm)


, "CARACTERISTIQUES": 
, "TYPE_MARCHE": 

, "RENSEIGNEMENTS_COMPLEMENTAIRES": 
"ADRESSE": 

, "CONDITION_PARTICIPATION": 

, "CONDITION_RELATIVE_MARCHE": 
, "TYPE_ORGANISME": 
, "PROCEDURE": 
, "CRITERES_ATTRIBUTION": 
"CRITERE": [
, 
]



In [12]:
# si un élément a plusieurs clés valeurs mais que l'element se termine par ':' on ajoute ""
for i in range(len(first_line)):
    elm_bis = first_line[i].split(",")
    temp=""
    for micro_elm in elm_bis:
        if re.search(':',micro_elm) and (not re.search(':("|\s)',micro_elm) and (': [' not in micro_elm)):
            print("micro : ",micro_elm)
            micro_elm += '""'
            print("micro change: ",micro_elm)
            temp+= micro_elm+","
        elif not re.search(':|"',micro_elm):
            temp+= micro_elm+ " "
        else:
            temp+= micro_elm+","            
    temp = temp[:-1] # on enlève la dernière virgule
    first_line[i] =  temp       

In [13]:
first_line

['"VARIANTES_NON": "", "LIEU_EXEC_LIVR": ',
 '"VILLE": "Pontivy", "CP": "56300", "ADRESSE": "Territoire de pontivy communauté"',
 '  "OBJET_COMPLET": "Réalisation d\'un schéma directeur des systèmes d\'information intercommunal", "DIV_EN_LOTS": ',
 '"NON": ""',
 '"QUANTITE": "Réalisation d\'un schéma directeur des systèmes d\'information intercommunal", "PRINCIPALES": "Délai d\'exécution : 36 mois Variantes non autorisées.Accord-cadre à bons de commande et à tranche optionnelle. Marché à prix mixtes : forfaitaires et unitaires à bons de commande.Pontivy Communauté figurant au titre des pilotes du projet national 100% demat des marchés publics, conformément à l\'article 40 du Décret 2016-360  la collectivité impose pour ce marché le mode de transmission des offres par voie électronique notamment via le dispositif MPS permettant de réduire les informations à transmettre. Ce marché fait partie du dispositif \\" marché public simplifié \\" (MPS) dans le cadre du programme national \\" Dite

In [14]:
dictio ={}
for elm in first_line:
    for elm_bis in elm.split(',', elm.count(": ")- elm.count("adresse suivante")):
        try:
            cle_val =elm_bis.split(': ')
            cle_dictio= re.sub('"','',cle_val[0])
            if cle_dictio.upper() == cle_dictio:
                cle_dictio = re.sub(' ','',cle_dictio)
            val_dictio = re.sub('"','',' '.join(cle_val[1:]))
            dictio[cle_dictio] = val_dictio
        except:
            print(elm_bis)
            print(elm_bis.split(':'))

In [15]:
dictio

{'VARIANTES_NON': '',
 'LIEU_EXEC_LIVR': '',
 'VILLE': 'Pontivy cédex',
 'CP': '56303',
 'ADRESSE': '1 Place Ernest Jan - bp96',
 'OBJET_COMPLET': "Réalisation d'un schéma directeur des systèmes d'information intercommunal",
 'DIV_EN_LOTS': '',
 'NON': '',
 'QUANTITE': "Réalisation d'un schéma directeur des systèmes d'information intercommunal",
 'PRINCIPALES': "Délai d'exécution  36 mois Variantes non autorisées.Accord-cadre à bons de commande et à tranche optionnelle. Marché à prix mixtes  forfaitaires et unitaires à bons de commande.Pontivy Communauté figurant au titre des pilotes du projet national 100% demat des marchés publics",
 " conformément à l'article 40 du Décret 2016-360  la collectivité impose pour ce marché le mode de transmission des offres par voie électronique notamment via le dispositif MPS permettant de réduire les informations à transmettre. Ce marché fait partie du dispositif \\ marché public simplifié \\ (MPS) dans le cadre du programme national \\ Dites-le-nous 

In [16]:
print(len(dictio.keys()))
dictio.keys()

39


dict_keys(['VARIANTES_NON', 'LIEU_EXEC_LIVR', 'VILLE', 'CP', 'ADRESSE', 'OBJET_COMPLET', 'DIV_EN_LOTS', 'NON', 'QUANTITE', 'PRINCIPALES', " conformément à l'article 40 du Décret 2016-360  la collectivité impose pour ce marché le mode de transmission des offres par voie électronique notamment via le dispositif MPS permettant de réduire les informations à transmettre. Ce marché fait partie du dispositif \\ marché public simplifié \\ (MPS) dans le cadre du programme national \\ Dites-le-nous une fois \\. La réponse papier ne sera pas acceptée.", 'TRAVAUX', 'SERVICE', 'IDENTITE', 'FAX', 'TEL', 'URL', 'MEL', 'CORRESPONDANT', ' Service marchés publics', 'DENOMINATION', 'RENS_COMPLEMENT', " les candidats doivent choisir un seul mode de transmission de leur offre  soit par voie électronique  soit par le dispositif MPS  sous peine du rejet de l'offre.Pour une remise des offres dématérialisée classique ", " et permet aux entreprises de candidater sur la base de leur seul numéro de SIRET. Le mode

In [17]:
liste_cle = list(dictio.keys())
for i in range(len(liste_cle)-1,-1,-1):
    if len(liste_cle[i]) >40 or "soit" in liste_cle[i] :
        dictio[liste_cle[i-1]] = dictio[liste_cle[i-1]] + liste_cle[i] +dictio[liste_cle[i]]
        del dictio[liste_cle[i]]

In [406]:
len(dictio.keys())

36

In [407]:
dictio.keys()

dict_keys(['VARIANTES_NON', 'LIEU_EXEC_LIVR', 'VILLE', 'CP', 'ADRESSE', 'OBJET_COMPLET', 'DIV_EN_LOTS', 'NON', 'QUANTITE', 'PRINCIPALES', 'TRAVAUX', 'SERVICE', 'IDENTITE', 'FAX', 'TEL', 'URL', 'MEL', 'CORRESPONDANT', ' Service marchés publics', 'DENOMINATION', 'RENS_COMPLEMENT', 'ADRESSES_COMPLEMENTAIRES', 'TYPE', 'DOCUMENTS', 'ELIGIBLE_MPS', 'oui', 'CONDITION_ADMINISTRATIVE', 'REFERENCE_MARCHE', 'FINANCEMENT', 'EPT', 'TYPE_PROCEDURE', 'PROCEDURE_ADAPTE', 'CRITERES_PONDERES', '#text', '@POIDS', ''])

In [18]:
dictio

{'VARIANTES_NON': '',
 'LIEU_EXEC_LIVR': '',
 'VILLE': 'Pontivy cédex',
 'CP': '56303',
 'ADRESSE': '1 Place Ernest Jan - bp96',
 'OBJET_COMPLET': "Réalisation d'un schéma directeur des systèmes d'information intercommunal",
 'DIV_EN_LOTS': '',
 'NON': '',
 'QUANTITE': "Réalisation d'un schéma directeur des systèmes d'information intercommunal",
 'PRINCIPALES': "Délai d'exécution  36 mois Variantes non autorisées.Accord-cadre à bons de commande et à tranche optionnelle. Marché à prix mixtes  forfaitaires et unitaires à bons de commande.Pontivy Communauté figurant au titre des pilotes du projet national 100% demat des marchés publics conformément à l'article 40 du Décret 2016-360  la collectivité impose pour ce marché le mode de transmission des offres par voie électronique notamment via le dispositif MPS permettant de réduire les informations à transmettre. Ce marché fait partie du dispositif \\ marché public simplifié \\ (MPS) dans le cadre du programme national \\ Dites-le-nous une f

In [409]:
try:
    del dictio['#text']
except:
    print('non')

In [410]:
liste_cle =list(dictio.keys())
for i in range(len(liste_cle)-1,-1,-1):
    if liste_cle[i].upper() != liste_cle[i]:
        print(liste_cle[i])
        dictio[liste_cle[i-1]] += liste_cle[i]
        del dictio[liste_cle[i]]

oui
 Service marchés publics


In [411]:
liste_cle =list(dictio.keys())
for k in liste_cle:
    if dictio[k]=='':
        del dictio[k]

In [412]:
dictio.keys()

dict_keys(['VILLE', 'CP', 'ADRESSE', 'OBJET_COMPLET', 'QUANTITE', 'PRINCIPALES', 'FAX', 'TEL', 'URL', 'MEL', 'CORRESPONDANT', 'DENOMINATION', 'RENS_COMPLEMENT', 'ELIGIBLE_MPS', 'REFERENCE_MARCHE', 'FINANCEMENT', '@POIDS'])

In [413]:
dictio

{'VILLE': 'Pontivy cédex',
 'CP': '56303',
 'ADRESSE': '1 Place Ernest Jan - bp96',
 'OBJET_COMPLET': "Réalisation d'un schéma directeur des systèmes d'information intercommunal",
 'QUANTITE': "Réalisation d'un schéma directeur des systèmes d'information intercommunal",
 'PRINCIPALES': "Délai d'exécution  36 mois Variantes non autorisées.Accord-cadre à bons de commande et à tranche optionnelle. Marché à prix mixtes  forfaitaires et unitaires à bons de commande.Pontivy Communauté figurant au titre des pilotes du projet national 100% demat des marchés publics conformément à l'article 40 du Décret 2016-360  la collectivité impose pour ce marché le mode de transmission des offres par voie électronique notamment via le dispositif MPS permettant de réduire les informations à transmettre. Ce marché fait partie du dispositif \\ marché public simplifié \\ (MPS) dans le cadre du programme national \\ Dites-le-nous une fois \\. La réponse papier ne sera pas acceptée.",
 'FAX': '02-97-25-63-69',
 